<a href="https://colab.research.google.com/github/foryoudrizzle14/AI-/blob/main/3%EC%A3%BC%EC%B0%A8_%EC%88%98%EC%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3주차 DNN

## XOR 문제

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [ ]:
linear = nn.Linear(2,1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear,
                      sigmoid).to(device)

In [ ]:
loss_fn = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [ ]:
def train_model(epoch):
    for step in range(epoch):
        pred = model(X)
        cost = loss_fn(pred, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if step % 1000 == 0:
          print(step, cost.item())



In [ ]:
train_model(10001)

0 0.7099932432174683
1000 0.6931471824645996
2000 0.6931471824645996
3000 0.6931471824645996
4000 0.6931471824645996
5000 0.6931471824645996
6000 0.6931471824645996
7000 0.6931471824645996
8000 0.6931471824645996
9000 0.6931471824645996
10000 0.6931471824645996


In [ ]:
def evaluation():
    model.eval()
    with torch.no_grad():
        pred = model(X)
        predicted = (pred > 0.5).float()
        accuracy = (predicted == Y).float().mean()
        print('모델의 출력값(Hypothesis): ', pred.detach().cpu().numpy())
        print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
        print('실제값(Y): ', Y.cpu().numpy())
        print('정확도(Accuracy): ', accuracy.item())

In [ ]:
evaluation()

모델의 출력값(Hypothesis):  [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted):  [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


DNN(XOR)

In [ ]:
model = nn.Sequential(nn.Linear(2, 10),
                      nn.Sigmoid(),
                      nn.Linear(10, 10),
                      nn.Sigmoid(),
                      nn.Linear(10, 10),
                      nn.Sigmoid(),
                      nn.Linear(10, 1),
                      nn.Sigmoid()
                      ).to(device)

In [ ]:
loss_fn = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [ ]:
train_model(10001)

0 0.6937591433525085
1000 0.6931439638137817
2000 0.693138062953949
3000 0.6931298971176147
4000 0.6931158304214478
5000 0.6930845975875854
6000 0.6929795145988464
7000 0.6916526556015015
8000 0.0025153213646262884
9000 0.0006270514568313956
10000 0.0003462415188550949


In [ ]:
evaluation()

모델의 출력값(Hypothesis):  [[2.6845955e-04]
 [9.9966311e-01]
 [9.9964714e-01]
 [4.2584070e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0


In [ ]:
step = 999

step % 1000 == 0

False

과제

In [ ]:
import torch
import torch.nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 프레임으로 변환
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 텐서로 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

클래스 > nn.Sequential로 바꾸기
or 클래스 구조 바꾸기(네트워크 바꾸기)

In [ ]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 50) # 입력 특성은 4개, 첫 번째 은닉층 노드는 50개
        self.fc2 = nn.Linear(50, 3) # 출력 클래스는 3개

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = IrisNet()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 학습 과정
epochs = 100
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 10/100, Loss: 0.2412
Epoch 20/100, Loss: 0.0030
Epoch 30/100, Loss: 0.0053
Epoch 40/100, Loss: 0.0058
Epoch 50/100, Loss: 0.0081
Epoch 60/100, Loss: 0.3678
Epoch 70/100, Loss: 0.0020
Epoch 80/100, Loss: 0.0004
Epoch 90/100, Loss: 0.0102
Epoch 100/100, Loss: 0.2065


Softmax 평가함수

3가지 종류의 Species를 분류

[0.5, 0.7, 0.1]

=> 가장 큰 값의 index

armax